In [ ]:
**In this notebook, I would like to predicting power fails in 10 days periods.**
===========================================
1. Generate a dataset for this task
2. Try out and evaluate models

In [1]:
#Import packages
import pytz
import pandas as pd
import numpy as np
from datetime import datetime, date, time, timedelta
import re
from IPython.display import Image
import itertools
from random import randint, sample

**Set up working directory**
==============================

In [2]:
#Set up Working Directory and Data Files
drive = 'G'
if drive == 'G':
    data_dir = '../01.data/'
    output_dir = '../05.outputs/'
else:
    data_dir = '..\\01.data\\'

**Read in events data (sms2.csv)**
===================================

In [3]:
#Read in observed data-processed
e = pd.read_csv(data_dir + 'smsV2.csv',parse_dates=['datetime_rcvd'])

e = e.sort_values (by=['device_id','datetime_rcvd'])

#Lets add a time stamp which truncates to hour
e['datetime_rcvd_hr'] = e['datetime_rcvd'].apply (lambda x: datetime (x.year, x.month, x.day, x.hour))

**Lets get the time bounds of events from the dataset**
=========================================================

In [4]:
start = e.datetime_rcvd_hr.min()
end = e.datetime_rcvd_hr.max()
delta = timedelta(hours = 1)
print ('Events run from :  %s  to :   %s ' %(start.ctime(),end.ctime()))

Events run from :  Wed Oct 19 10:00:00 2016  to :   Thu Mar 23 16:00:00 2017 


**Create an a blank hourly data frame to capture events at each hour**
======================================================================
The idea is that I will fill/link this with actual events (dataframe e)

**Attempt to use a multi-index**
===============================

In [5]:
dates = [start]
step = timedelta(hours=1)

lst_boxes = list (e.device_id.unique())

while start <= end:
    start += step
    dates.append(start)

tuples = list(itertools.product(*[lst_boxes,dates]))

print ('Number of date-hour-box pair is %s' %len(dates))

idx = pd.MultiIndex.from_tuples(tuples, names=['device_id', 'datetime_rcvd_hr'])

#Ideal hourly dataframe would look like this: eb should mean events blank
eb = pd.DataFrame(index=idx)

eb = eb.sort_index()

eb = eb.reset_index()

Number of date-hour-box pair is 3728


In [6]:
eb.head()

device_id    datetime_rcvd_hr
0       1001 2016-10-19 10:00:00
1       1001 2016-10-19 11:00:00
2       1001 2016-10-19 12:00:00
3       1001 2016-10-19 13:00:00
4       1001 2016-10-19 14:00:00

In [7]:
#Merge the two datasets
ev = pd.merge(left=eb,right=e, on=['device_id','datetime_rcvd_hr'],how='left')

In [8]:
ev.head()

device_id    datetime_rcvd_hr datetime_rcvd datetime_sent body  msg  \
0       1001 2016-10-19 10:00:00           NaT           NaN  NaN  NaN   
1       1001 2016-10-19 11:00:00           NaT           NaN  NaN  NaN   
2       1001 2016-10-19 12:00:00           NaT           NaN  NaN  NaN   
3       1001 2016-10-19 13:00:00           NaT           NaN  NaN  NaN   
4       1001 2016-10-19 14:00:00           NaT           NaN  NaN  NaN   

   msg_num  
0      NaN  
1      NaN  
2      NaN  
3      NaN  
4      NaN

**Clean up events to ensure that for each box, time is bounded by time of actual events**
=========================================================================================
This ensures that we nevr extrapolate (forecast).....

In [9]:
#We add the 
ev2 = pd.DataFrame (columns=ev.columns)

for bx in list(ev.device_id.unique()):
    
    df_bx = ev[ev.device_id==bx]
        
    last_event = df_bx.datetime_rcvd.max()
    ts_upper = datetime (last_event.year, last_event.month, last_event.day, last_event.hour)
    
    
    first_event = df_bx.datetime_rcvd.min()
    ts_lower = datetime (first_event.year, first_event.month, first_event.day, first_event.hour)
    
    mask = (df_bx['datetime_rcvd_hr'] >= ts_lower) & (df_bx['datetime_rcvd_hr'] <= ts_upper)
    
    df_bx = df_bx.loc[mask]
    
    ev2 = ev2.append(df_bx)

#Replace ev with ev2
ev = ev2

In [10]:
ev2.head()

device_id    datetime_rcvd_hr       datetime_rcvd        datetime_sent  \
185     1001.0 2016-10-27 03:00:00 2016-10-27 03:12:55  2016-10-27 03:13:20   
186     1001.0 2016-10-27 04:00:00                 NaT                  NaN   
187     1001.0 2016-10-27 05:00:00                 NaT                  NaN   
188     1001.0 2016-10-27 06:00:00                 NaT                  NaN   
189     1001.0 2016-10-27 07:00:00                 NaT                  NaN   

                                                  body        msg  msg_num  
185  Power Failure  is running ID 1001 | Monitoring...  pfail_mon      3.0  
186                                                NaN        NaN      NaN  
187                                                NaN        NaN      NaN  
188                                                NaN        NaN      NaN  
189                                                NaN        NaN      NaN

**Lets deal with status of the box**
======================================
1. Alive (1) - when the box gets monitoring messages (at least 2 in 24 hours)

2. Dead (0) - When box gets less than 2　messages in 24  hours

So box status is correct only up to 12 hours.

In [12]:
#Add column to indicate whether box is sending ping messages or not
#The two messages to indicate box life are: 'pfail_mon' or 'pon_mon'

ev['ping_msg'] = 0

ev.ix[ev.msg=='pfail_mon',['ping_msg']] =  1

ev.ix[ev.msg=='pon_mon',['ping_msg']] =  1

ev = ev.sort_values (by = ['datetime_rcvd_hr'])

In [13]:
ev.columns

Index(['device_id', 'datetime_rcvd_hr', 'datetime_rcvd', 'datetime_sent',
       'body', 'msg', 'msg_num', 'ping_msg'],
      dtype='object')

**Now we add a box_state column**
=====================================

In [14]:
#Initialise box_state to 0
ev['box_state'] = 0

ev = ev.sort_values (by = ['device_id','datetime_rcvd_hr'])

start = ev.datetime_rcvd_hr.min()

end_all = ev.datetime_rcvd_hr.max()

step = timedelta (hours = 24)

while start <= end_all:
    end = start + step
    
    mask = (ev['datetime_rcvd_hr'] >= start) & (ev['datetime_rcvd_hr'] <= end)
    
    check_sum = ev.loc[mask].ping_msg.sum()
    
    
    if check_sum >= 2:
        ev.ix[mask,'box_state'] = 1
    else:
        ev.ix[mask,'box_state'] = 0
    
   
    start += step

**Now lets deal with power-state**
===================================
1. 1-power on
2. 0-power off
3. -1-power status unknown

In [15]:
#Initialise power-state to -1
ev['pwr_state'] = np.nan

#To indicate source of pwr_state information
ev['pwr_state_src'] = ''

#When message is pon_monitoring or power back, set power status to 1
on_mask = ev['msg'].isin(['pback','pon_mon'])

ev.ix[on_mask,'pwr_state'] = 1

ev.ix[on_mask,'pwr_state_src'] = 'actual'

off_mask = ev['msg'].isin(['pfail','pfail_mon'])

ev.ix[off_mask,'pwr_state'] = 0

ev.ix[off_mask,'pwr_state_src'] = 'actual'

#Set source of message as actual or interpolated

In [16]:
ev.columns


Index(['device_id', 'datetime_rcvd_hr', 'datetime_rcvd', 'datetime_sent',
       'body', 'msg', 'msg_num', 'ping_msg', 'box_state', 'pwr_state',
       'pwr_state_src'],
      dtype='object')

In [17]:
#Just interpolating events to make sure the matrix isnt sparse
def interpolate_pwr_state (df,t0,t1,e0,max_intv):
        #Deduce power state from message
        #Lets check duration of this period
        diff_hrs = ((t1-t0).total_seconds())/3600
        
        #print (t1.ctime(),'--',t1.ctime())
        #print ('Duration between events is %s hours'%diff_hrs)
        mask = (df['datetime_rcvd_hr'] >= t0) & (df['datetime_rcvd_hr'] <= t1)
        ping_msgs = df.loc[mask].ping_msg.sum()
        #print (ping_msgs)
        
        #if duration is <=24 hours and number of monitoring messages is >= 2 interpolate pwr_state
        
        if diff_hrs <= max_intv:
            #scenario one poff
            if ping_msgs>=diff_hrs/12:
                if e0 == 'pfail' or e0 == 'pfail_mon':
                    return 0
                elif e0 == 'pon_mon' or e0 == 'pback':
                    return 1
            else:
                return -1
        else:
            return -1

In [19]:
ev.head()

device_id    datetime_rcvd_hr       datetime_rcvd        datetime_sent  \
185     1001.0 2016-10-27 03:00:00 2016-10-27 03:12:55  2016-10-27 03:13:20   
186     1001.0 2016-10-27 04:00:00                 NaT                  NaN   
187     1001.0 2016-10-27 05:00:00                 NaT                  NaN   
188     1001.0 2016-10-27 06:00:00                 NaT                  NaN   
189     1001.0 2016-10-27 07:00:00                 NaT                  NaN   

                                                  body        msg  msg_num  \
185  Power Failure  is running ID 1001 | Monitoring...  pfail_mon      3.0   
186                                                NaN        NaN      NaN   
187                                                NaN        NaN      NaN   
188                                                NaN        NaN      NaN   
189                                                NaN        NaN      NaN   

     ping_msg  box_state  pwr_state pwr_state_src  
185         1          1        0.0        actual  
186         0          1        NaN                
187         0          1        NaN                
188         0          1        NaN                
189         0          1        NaN

In [20]:
#Just interpolating events to make sure the matrix isnt sparse
def interpolate_events (df, power_state, max_hrs):
    
    df = df.sort_values (by =['datetime_rcvd_hr'])
    
    
    events = df[['datetime_rcvd_hr','msg']]
    
    #extract only events-hopefully they are still in sorted order
    events = events.dropna(axis=0, how='any')
    
    #index of events
    idx = events.index
    
    #a counter 
    i = 0
    
    #Loop through all events (2 at a time) and extract corresponding rows in df
    while i < len(idx)-1:
 
        t0 = events.ix[idx[i],'datetime_rcvd_hr']
      
        t1 = events.ix[idx[i+1],'datetime_rcvd_hr']
        
        diff_hrs = ((t1-t0).total_seconds())/3600
        
        #Event at start hour
        ev0 = events.ix[idx[i],'msg']
        
        #Event at end hour
        ev1 = events.ix[idx[i+1],'msg']
        
        #Rows in df matching events in this interval
        mask = (df['datetime_rcvd_hr'] > t0) & (df['datetime_rcvd_hr'] < t1)
        #print (len(mask[mask==True]))
        

        #Set all events in the range to ev0
        df.ix[mask,'msg'] = ev0
        
        #Set source as 'interpolated'
        df.ix[mask,'msg_src'] = 'interpolated'
        
        #Also interpolate power state
        if power_state:
            #increment counter
            pwr = interpolate_pwr_state (df,t0,t1,ev0,max_hrs)
            #set power-state to return value
            #Set all events in the range to ev0
            df.ix[mask,'pwr_state'] = pwr
        
            #Set source as 'interpolated'
            df.ix[mask,'pwr_state_src'] = 'interpolated'
            
        i +=1
    
    
    return df

**Now interpolate events and label them as such**
=================================================
The method being used is simple:
The time slots x such that x > a and x < b 
between two events (a,b) takes the event
a.

In [24]:
#Label actual events as 'actual
#and rest of the events as interpolated
actual_msgs = ev.loc[ev.msg.notnull()].index

ev.ix[actual_msgs, 'msg_src'] = 'actual'

boxes = [int(i) for i in list (ev.device_id.unique())]

#Load new data into this empty dataframe
evi = pd.DataFrame (columns=ev.columns)

for i, b in enumerate(boxes[:5]):
    if i%25 == 0:
        print ('Working on box %s of %s'%(i,len(boxes)))
    
    df_bx = ev[ev.device_id==b]
    
    evi = evi.append(interpolate_events (df_bx,True, 48))

Working on box 0 of 300


In [27]:
#Add box details
bx = pd.read_csv(data_dir + 'Boxes.csv',parse_dates=['DateCollectionStart'])

bx = bx[['ClusterId','DateCollectionStart','LONG','LAT', 'BoxID']]

bx.rename(columns={'ClusterId': 'psu','LONG':'lon','LAT':'lat','BoxID':'device_id'},inplace=True)

In [28]:
#Merge with evi:
evi = evi.sort_values(by = ['device_id','datetime_rcvd_hr'])

evi2 = pd.merge (left = evi, right = bx, on = 'device_id', how = 'left')

In [29]:
evi2.head()

device_id    datetime_rcvd_hr       datetime_rcvd        datetime_sent  \
0     1001.0 2016-10-27 03:00:00 2016-10-27 03:12:55  2016-10-27 03:13:20   
1     1001.0 2016-10-27 04:00:00                 NaT                  NaN   
2     1001.0 2016-10-27 05:00:00                 NaT                  NaN   
3     1001.0 2016-10-27 06:00:00                 NaT                  NaN   
4     1001.0 2016-10-27 07:00:00                 NaT                  NaN   

                                                body        msg  msg_num  \
0  Power Failure  is running ID 1001 | Monitoring...  pfail_mon      3.0   
1                                                NaN  pfail_mon      NaN   
2                                                NaN  pfail_mon      NaN   
3                                                NaN  pfail_mon      NaN   
4                                                NaN  pfail_mon      NaN   

   ping_msg  box_state  pwr_state pwr_state_src       msg_src  psu  \
0       1.0        1.0        0.0        actual        actual   10   
1       0.0        1.0        0.0  interpolated  interpolated   10   
2       0.0        1.0        0.0  interpolated  interpolated   10   
3       0.0        1.0        0.0  interpolated  interpolated   10   
4       0.0        1.0        0.0  interpolated  interpolated   10   

  DateCollectionStart        lon        lat  
0          2016-11-25  68.763315  38.517016  
1          2016-11-25  68.763315  38.517016  
2          2016-11-25  68.763315  38.517016  
3          2016-11-25  68.763315  38.517016  
4          2016-11-25  68.763315  38.517016

**Determine whether one box can predict the other in one cluster**
====================================================================

In [444]:
#A utility function to compare 
def compare (row,col):
    if row[col + '_bx1'] == row[col + '_bx2']:
        return 1
    else:
        return 0

In [504]:
evi2.msg.unique()

array(['pfail_mon', 'pfail', 'pback', 'pon_mon'], dtype=object)

In [567]:
def random_events ():
    i = randint(0,3)
    ev = ['pfail_mon', 'pfail', 'pback', 'pon_mon']
    ev = random.sample(ev, len(ev)) 
    
    return ev[i]

In [502]:

def evaluate_nearest_box_predictor(bx1, bx2):
    #keep only actual events
    bx1 = bx1[bx1.msg_src=='actual']
    bx2 = bx2[bx2.msg_src=='actual']
        
    #Lets count equal events (messages)
    use_cols = ['device_id','msg','datetime_rcvd_hr','datetime_rcvd']
    bx1 = bx1[use_cols]
    bx2 = bx2[use_cols]
        
    #Merge-keep only events which are common
    bx12 = pd.merge(left=bx1,right=bx2, on = 'datetime_rcvd_hr', how='inner', suffixes=['_bx1','_bx2'])
        
    bx12['compare_msgs'] = bx12.apply(lambda x: compare(x,'msg'), axis=1)
        
    #Return proportion of equal events as accuracy
    val_cnts = bx12.compare_msgs.value_counts()

    acc = (val_cnts[1]/bx12.shape[0])*100
        
    
    return acc

In [593]:
#join the data of the two boxes based on same time
#and compare events on those times
def evaluate_nearest_box_predictor2(bx1, bx2, rand):
    #keep only actual events
    bx1 = bx1[bx1.msg_src=='actual']
    bx2 = bx2[bx2.msg_src=='actual']
    
    if rand:
        #Lets count equal events (messages)
        use_cols = ['device_id','msg','datetime_rcvd_hr','datetime_rcvd']
        bx1 = bx1[use_cols]
        bx2 = bx2[use_cols]
        
        #Replace bx2 with random messages
        #+++++++++++++++++++++++++++++++++++++++ 
        bx2['msg'] = bx2.apply(lambda x: random_events(),axis=1)
       
        #Merge-keep only events which are common
        bx12 = pd.merge(left=bx1,right=bx2, on = 'datetime_rcvd_hr', how='inner', suffixes=['_bx1','_bx2'])

        bx12['compare_msgs'] = bx12.apply(lambda x: compare(x,'msg'), axis=1)
        
        #Return proportion of equal events as accuracy
        val_cnts = bx12.compare_msgs.value_counts()

        acc_1 = (val_cnts[1]/bx12.shape[0])*100
        
        #+++++++++++++++++++++++++++++++++++++++ 
        bx1['msg'] = bx1.apply(lambda x: random_events(),axis=1)

        #Merge-keep only events which are common
        bx12 = pd.merge(left=bx1,right=bx2, on = 'datetime_rcvd_hr', how='inner', suffixes=['_bx1','_bx2'])

        bx12['compare_msgs'] = bx12.apply(lambda x: compare(x,'msg'), axis=1)

        #Return proportion of equal events as accuracy
        val_cnts = bx12.compare_msgs.value_counts()

        acc_2 = (val_cnts[1]/bx12.shape[0])*100
        
        return np.max([acc_2, acc_1])
    
    else:
        #Lets count equal events (messages)
        use_cols = ['device_id','msg','datetime_rcvd_hr','datetime_rcvd']
        bx1 = bx1[use_cols]
        bx2 = bx2[use_cols]

        #Merge-keep only events which are common
        bx12 = pd.merge(left=bx1,right=bx2, on = 'datetime_rcvd_hr', how='inner', suffixes=['_bx1','_bx2'])

        bx12['compare_msgs'] = bx12.apply(lambda x: compare(x,'msg'), axis=1)

        #Return proportion of equal events as accuracy
        val_cnts = bx12.compare_msgs.value_counts()

        acc = (val_cnts[1]/bx12.shape[0])*100


        return acc

**We now consider each cluster and compute accuracy (proportion of events which match)**

In [594]:
lst_psu = list(evi2.psu.unique())

tot = len(lst_psu)
acc_all = []
for p in lst_psu:
    try:
        evi_p = evi2[evi2.psu==p]
    
        boxes = list(evi_p.device_id.unique())
    
        if len(boxes)>2:
            #print ('Boxes: %s'%len(boxes))
            tot -= 1
            continue
        
        bx1,bx2 = evi_p[evi_p.device_id==boxes[0]], evi_p[evi_p.device_id==boxes[1]]
    
        acc = evaluate_nearest_box_predictor2(bx1,bx2,False)
        #print (acc)
        acc_all.append(acc)
    except Exception as e:
        #print(e)
        pass
    
    #print ('PSU-%s, accuracy: %s'%(p,acc))

print ('Total Clusters Evaluated: %s'%tot)
print ('Accuracy (%)-summary across all boxes : ')
print ('Worst case: %s'%(np.min(acc_all)),
       ' | Best case : %s'%(np.max(acc_all)),
       ' | Median : %s'%(np.median(acc_all)),' | Std deviation : %s'%(np.std(acc_all)))

Total Clusters Evaluated: 137
Accuracy (%)-summary across all boxes : 
Worst case: 3.125  | Best case : 100.0  | Median : 57.4854558127  | Std deviation : 15.9750785311


**Now lets see how a random predictor would do...........**

In [595]:
lst_psu = list(evi2.psu.unique())

tot = len(lst_psu)
acc_all = []
for p in lst_psu:
    try:
        evi_p = evi2[evi2.psu==p]
    
        boxes = list(evi_p.device_id.unique())
    
        if len(boxes)>2:
            #print ('Boxes: %s'%len(boxes))
            tot -= 1
            continue
        
        bx1,bx2 = evi_p[evi_p.device_id==boxes[0]], evi_p[evi_p.device_id==boxes[1]]
    
        acc = evaluate_nearest_box_predictor2(bx1,bx2,True)
        #print (acc)
        acc_all.append(acc)
    except Exception:
        pass
    
    #print ('PSU-%s, accuracy: %s'%(p,acc))

print ('Total Clusters Evaluated: %s'%tot)
print ('Accuracy (%)-summary across all boxes : ')
print ('Worst case: %s'%(np.min(acc_all)),
       ' | Best case : %s'%(np.max(acc_all)),
       ' | Median : %s'%(np.median(acc_all)),' | Std deviation : %s'%(np.std(acc_all)))

Total Clusters Evaluated: 137
Accuracy (%)-summary across all boxes : 
Worst case: 12.5  | Best case : 100.0  | Median : 26.3157894737  | Std deviation : 9.48622241919


**So,  using events from another box seem to do better than random guessing of events**
========================================================================================

In [ ]:
def get_event_freq (df, t0, t1, by_box, cond):
    """
    This function returns a key value pair where the key is hour of the day
    and value is another key-value pair of event and frequencies

    @param df: reasonably preprocessed data
    @param boxes: boxes to use to compute the freqs
    @param t0,t1: Time interval to compute the events in
    @param cond: e.g, consider weekday events only (To be implemented later)
    @return: Returns a key value pair where the key is hour of the day
    and value is another key-value pair of event and frequencies
    @raise keyError: raises an exception
    """
    mask = (df['datetime_rcvd_hr'] > t0) & (df['datetime_rcvd_hr'] < t1)
    
    #Events
    df = df.loc[mask]
     
    if by_box:
        #Add hour column
        df['hr'] = df['datetime_rcvd_hr'].apply(lambda x: x.hour)

    else:
        
        df['hr'] = df['datetime_rcvd_hr'].apply(lambda x: x.hour)
        
        hr_cnts = df.groupby(['hr','msg'])['msg'].agg(['count'])
       
        hr_cnts = hr_cnts.reset_index()
            
    
    return hr_cnts

In [ ]:
def moving_time_window_predictor(train_data, pred_date, window_length):
    t1 = pred_date - timedelta(days=window_length)
    
    t0 = pred_date
   
    t2 = pred_date + timedelta(days=window_length)
   
    freqs = get_event_freq (train_data, t1, t2, False, False)
    
    pred_hr = pred_date.hour
    
    events_hr = freqs[freqs.hr==pred_hr]
    
    return events_hr.max(axis=0).msg

**Lets evaluate how a simple window based predictor would do....**
This works loosely on the idea of periodicity in the data.

In [ ]:
def evaluate_temporal_window_predictor(bx, test_id, prop_test, window):
    test_bx = bx[bx.device_id==test_id]
    
    event_dates = list(test_bx.datetime_rcvd_hr)

    num_tests = int (test_bx.shape[0]*prop_test)
    
        
    correct = 0
    tot = 0
    for i in range(num_tests):
        test_date = event_dates[randint(0,num_tests)]

        
        #They are multiple events in some cases-only pick first one
        actual_events = test_bx[test_bx.datetime_rcvd_hr==test_date].msg.values
        
        actual = actual_events[0]
        
        #Remove this row from trainign data
        train = bx[bx.datetime_rcvd_hr!=test_date]
        #print (bx.shape[0], '|', train.shape[0],'|', len(actual_events))   
        
        #predict
        predicted = moving_time_window_predictor(train,test_date,window)
            
        if predicted == actual:
            correct += 1
            
        tot +=1
        
    return correct/tot*100

In [ ]:
#use only actual events for this
evi2_a = evi2[evi2.msg_src=='actual']

lst_psu = list(evi2_a.psu.unique())

tot = len(lst_psu)
acc_all = []
for p in lst_psu[:5]:
    try:
        evi_p = evi2_a[evi2_a.psu==p]
        
        boxes = list(evi_p.device_id.unique())
    
        if len(boxes)>2:
            #print ('Boxes: %s'%len(boxes))
            tot -= 1
            continue
        
        acc1 = evaluate_temporal_window_predictor(evi_p, boxes[0] , 0.25, 7)
        
        acc2 = evaluate_temporal_window_predictor(evi_p, boxes[1] , 0.25, 7)
        
        acc_all = acc_all + [acc1,acc2]
    except Exception:
        pass
    
#print ('PSU-%s, accuracy: %s'%(p,acc))

print ('Total Clusters Evaluated: %s'%tot)
print ('Accuracy (%)-summary across all boxes : ')
print ('Worst case: %s'%(np.min(acc_all)),
       ' | Best case : %s'%(np.max(acc_all)),
       ' | Median : %s'%(np.median(acc_all)),' | Std deviation : %s'%(np.std(acc_all)))